In [ ]:
# Imp libs:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Spliting data
from sklearn.model_selection import train_test_split,GridSearchCV

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
# Importing metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Removing warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
file_loc = 'https://raw.githubusercontent.com/dsrscientist/DSData/master/Telecom_customer_churn.csv'
df = pd.read_csv(file_loc)

In [ ]:
print('First 10 rows')
df.head(10)

In [ ]:
print('Last 10 rows')
df.tail(10)

In [ ]:
df.shape

In [ ]:
custIDsplit=[]
custID1=[]
custID2=[]

for i in df.customerID:
    custIDsplit.append(i.split('-'))

In [ ]:
custIDsplit

In [ ]:
len(custIDsplit)

In [ ]:
for i in range(0,len(custIDsplit)):
    custID1.append(custIDsplit[i][0])
    
len(custID1)

In [ ]:
for i in range(0,len(custIDsplit)):
    custID2.append(custIDsplit[i][1])
    
len(custID2)

In [ ]:
df['CustID 2ndry']=custID2

In [ ]:
df['CustID Initials']=custID1

In [ ]:
df['CustID Initials']=df['CustID Initials'].astype('float64')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
null_data = df.isnull().sum()
null_data.sort_values(ascending=0)

In [ ]:
df.dtypes

In [ ]:
df.drop(df.loc[df['TotalCharges']==' '].index,inplace=True)

In [ ]:
df.shape

In [ ]:
df['TotalCharges'] = df['TotalCharges'].astype('float64')

In [ ]:
desc = df.describe().T
desc['range']=desc['max']-desc['min']
desc

In [ ]:
df.describe(include='object').T

In [ ]:
desc[['min','mean','max','range']]

In [ ]:
cont_data = df.select_dtypes(include=['int64','float64'])

cat_data= df.select_dtypes(include=['object'])

cont_columns = cont_data.columns

cat_columns = cat_data.columns

In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
from scipy import stats

for i in cat_columns:
    print('For',i,', most frequent value is: ',stats.mode(df[i]),'\n')

In [ ]:
for i in cat_columns:
    print('For column',i,'unique values are: ',df[i].unique())
    print('For column',i,'count of unique values are: ',df[i].nunique(),'\n\n')
    

In [ ]:
for i in cat_columns:
    print('For column --',i,'-- value counts are: \n',df[i].value_counts(),'\n\n')

In [ ]:
"""EXPLORATORY DATA ANALYSIS

1.
The data set contains 743 rows and 21 columns.

From these 21 columns only three columns are of numerical data types and rest are of object data types. 

customerID            object
gender                object
SeniorCitizen          int64
Partner               object
Dependents            object
tenure                 int64
PhoneService          object
MultipleLines         object
InternetService       object
OnlineSecurity        object
OnlineBackup          object
DeviceProtection      object
TechSupport           object
StreamingTV           object
StreamingMovies       object
Contract              object
PaperlessBilling      object
PaymentMethod         object
MonthlyCharges       float64
TotalCharges         float64
Churn                  int64
Avg Charges Range     object
CustID Initials      float64
CustID 2ndry          object

There are null values in the data set.

2.ANALYSING SUMMARY DESCRIPTIVE STATISTICS OF THE DATA

From the senior citizen column the minimum and maximum values are zero and one respectively that indicate whether a person 
is senior citizen or not.
From tenure column n the range is between 1 and 72 that are minimum and maximum values. The mean value of tenure is 32.42 
and standard deviation is 24.54 that is quite high.
Monthly charges start from 18.25 and maximum value is 118.75. the mean and standard deviation values are 64.79 and 30.08 
respectively. Total charges colum values very a lot that indicate presence of outliers.

3.ANALYSING COLUMNS CONTAINING OBJECT DATA TYPES:

Top frequency is of male gender where in the partner is no. 
For most people they have no dependents. For columns phone service and paperless billing most frequent answers are yes. 
The type of internet services fibre optic and contract type is month to month where and for most people payment method is 
electronic check type. 
For columns multiple lines online security online backup device protection tax support streaming TV streaming movies and 
values of churn the answers are no.


4. Column Selection and Feature Engineering

SELECTING COLUMNS CONTAINING CONTINUOUS DATA TYPES:

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

SELECTING COLUMNS CONTAINING CATEGORICAL DATA TYPES

Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'],
      dtype='object')
      
5. Univariate analysis:

Mode values:

        For customerID , most frequent value is:  ModeResult(mode=array(['0002-ORFBO'], dtype=object), count=array([1])) 

        For gender , most frequent value is:  ModeResult(mode=array(['Male'], dtype=object), count=array([3549])) 

        For Partner , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3639])) 

        For Dependents , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([4933])) 

        For PhoneService , most frequent value is:  ModeResult(mode=array(['Yes'], dtype=object), count=array([6352])) 

        For MultipleLines , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3385])) 

        For InternetService , most frequent value is:  ModeResult(mode=array(['Fiber optic'], dtype=object), count=array([3096])) 

        For OnlineSecurity , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3497])) 

        For OnlineBackup , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3087])) 

        For DeviceProtection , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3094])) 

        For TechSupport , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([3472])) 

        For StreamingTV , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([2809])) 

        For StreamingMovies , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([2781])) 

        For Contract , most frequent value is:  ModeResult(mode=array(['Month-to-month'], dtype=object), count=array([3875])) 

        For PaperlessBilling , most frequent value is:  ModeResult(mode=array(['Yes'], dtype=object), count=array([4168])) 

        For PaymentMethod , most frequent value is:  ModeResult(mode=array(['Electronic check'], dtype=object), count=array([2365])) 

        For Churn , most frequent value is:  ModeResult(mode=array(['No'], dtype=object), count=array([5163])) 


ANALYSING UNIQUE VALUES IN CATEGORICAL DATA

        For column customerID unique values are:  ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
         '3186-AJIEK']
        For column customerID count of unique values are:  7032 


        For column gender unique values are:  ['Female' 'Male']
        For column gender count of unique values are:  2 


        For column Partner unique values are:  ['Yes' 'No']
        For column Partner count of unique values are:  2 


        For column Dependents unique values are:  ['No' 'Yes']
        For column Dependents count of unique values are:  2 


        For column PhoneService unique values are:  ['No' 'Yes']
        For column PhoneService count of unique values are:  2 


        For column MultipleLines unique values are:  ['No phone service' 'No' 'Yes']
        For column MultipleLines count of unique values are:  3 


        For column InternetService unique values are:  ['DSL' 'Fiber optic' 'No']
        For column InternetService count of unique values are:  3 


        For column OnlineSecurity unique values are:  ['No' 'Yes' 'No internet service']
        For column OnlineSecurity count of unique values are:  3 


        For column OnlineBackup unique values are:  ['Yes' 'No' 'No internet service']
        For column OnlineBackup count of unique values are:  3 


        For column DeviceProtection unique values are:  ['No' 'Yes' 'No internet service']
        For column DeviceProtection count of unique values are:  3 


        For column TechSupport unique values are:  ['No' 'Yes' 'No internet service']
        For column TechSupport count of unique values are:  3 


        For column StreamingTV unique values are:  ['No' 'Yes' 'No internet service']
        For column StreamingTV count of unique values are:  3 


        For column StreamingMovies unique values are:  ['No' 'Yes' 'No internet service']
        For column StreamingMovies count of unique values are:  3 


        For column Contract unique values are:  ['Month-to-month' 'One year' 'Two year']
        For column Contract count of unique values are:  3 


        For column PaperlessBilling unique values are:  ['Yes' 'No']
        For column PaperlessBilling count of unique values are:  2 


        For column PaymentMethod unique values are:  ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
         'Credit card (automatic)']
        For column PaymentMethod count of unique values are:  4 


        For column Churn unique values are:  ['No' 'Yes']
        For column Churn count of unique values are:  2 

ANALYSING VALUE COUNTS OF COLUMNS CONTAINING CATEGORICAL DATAS

        For column -- customerID -- value counts are: 
         7590-VHVEG    1
        0265-PSUAE    1
        2956-GGUCQ    1
        6008-NAIXK    1
        5956-YHHRX    1
                     ..
        7874-ECPQJ    1
        9796-MVYXX    1
        2637-FKFSY    1
        1552-AAGRX    1
        3186-AJIEK    1
        Name: customerID, Length: 7032, dtype: int64 


        For column -- gender -- value counts are: 
         Male      3549
        Female    3483
        Name: gender, dtype: int64 


        For column -- Partner -- value counts are: 
         No     3639
        Yes    3393
        Name: Partner, dtype: int64 


        For column -- Dependents -- value counts are: 
         No     4933
        Yes    2099
        Name: Dependents, dtype: int64 


        For column -- PhoneService -- value counts are: 
         Yes    6352
        No      680
        Name: PhoneService, dtype: int64 


        For column -- MultipleLines -- value counts are: 
         No                  3385
        Yes                 2967
        No phone service     680
        Name: MultipleLines, dtype: int64 


        For column -- InternetService -- value counts are: 
         Fiber optic    3096
        DSL            2416
        No             1520
        Name: InternetService, dtype: int64 


        For column -- OnlineSecurity -- value counts are: 
         No                     3497
        Yes                    2015
        No internet service    1520
        Name: OnlineSecurity, dtype: int64 


        For column -- OnlineBackup -- value counts are: 
         No                     3087
        Yes                    2425
        No internet service    1520
        Name: OnlineBackup, dtype: int64 


        For column -- DeviceProtection -- value counts are: 
         No                     3094
        Yes                    2418
        No internet service    1520
        Name: DeviceProtection, dtype: int64 


        For column -- TechSupport -- value counts are: 
         No                     3472
        Yes                    2040
        No internet service    1520
        Name: TechSupport, dtype: int64 


        For column -- StreamingTV -- value counts are: 
         No                     2809
        Yes                    2703
        No internet service    1520
        Name: StreamingTV, dtype: int64 


        For column -- StreamingMovies -- value counts are: 
         No                     2781
        Yes                    2731
        No internet service    1520
        Name: StreamingMovies, dtype: int64 


        For column -- Contract -- value counts are: 
         Month-to-month    3875
        Two year          1685
        One year          1472
        Name: Contract, dtype: int64 


        For column -- PaperlessBilling -- value counts are: 
         Yes    4168
        No     2864
        Name: PaperlessBilling, dtype: int64 


        For column -- PaymentMethod -- value counts are: 
         Electronic check             2365
        Mailed check                 1604
        Bank transfer (automatic)    1542
        Credit card (automatic)      1521
        Name: PaymentMethod, dtype: int64 


        For column -- Churn -- value counts are: 
         No     5163
        Yes    1869
        Name: Churn, dtype: int64 


For gender values of male and female the ratio contribution of both is quite the same.
Out of total 4933 customers have no dependents. Phone service column data is biased and for most of the customers 
they have phone service.
Asparas internet services concerned first comes the fibre optic then the DSL and then the no answer is selected.
Churn customer data is also skewed as for most of the customers have been retained and are not churned.

STEP 6.BIVARIATE ANALYSIS 

USING CAT PLOT

        Most of the customers got turned and retained when they used electronic check as payment method.

        For rest of the payment methods very less customers got turned as compared to those who were retained.

        Mor customers got turned who choose yes for paperless billing.

        Lots of customers got turned whose contract was on month to month basis and least customers got turn who 
        is contract was for 2 years.

        Very less customerss got churned who obtained for no internet service as compared to those who obtained 
        for streaming movies and streaming TV Intex support and device protection.

        Very less customers got churned who opted for no internet service in case of online backup and online security.

        Huge percentage of customers got journed who obted for fibre optic in case of the type of internet service 
        as compared to DSL.

        The number of customers are very less when they opted for no phone service in case of multiple lines. 
        The number of customers who got turned is the same whether yes or not they opted for multiple lines.

        The number of accounts for both child and not joint for phone service is very less as compared to the 
        customers who opted for phone service.

        The customers who had depends got less as compared to those customers who didn't had any depends and the 
        count values also less.

        Customers got churned a little more when they had no partners.

        For both genders turn customers percentage seems equal.

        By average charges range most of the customers who got retained belong to those who had average charges 
        that is between 2000 to 6000 range.

ANALYSING BOX PLOTS

        Except credit card and bank transfer rush payment methods contain lots of outliers.

        Customers who opted for paperless billing and didn't got churned have no outliers.

        Customers having one year and 2 year contract do not contribute to outliers as oppose to the 
        customers having month to month contract.

        Customers who don't have streaming movies and got turned contribute most to outliers.

        Customers having no streaming TV and got turned contain most of outliers.

        Most of the outliers contribution is by customers who have no tax support no device protection 
        no online backup no online security and they got churned.

        Customers having different type of internet service contain outliers.

        Customers who didn't opted for multiple lines and had no phone service contributed to most of outliers.

        Customers who whether opted or not update the phone service got most of outliers in the churned category.

USING VIOLIN PLOT TO CHECK THE PEAK VALUES OF JOINT CUSTOMERS FOR DIFFERENT CONTINUOUS COLUMNS

        For total charges column since maximum density lies between rains 0 to 2000 peaks for both churned and not 
        churned customers lie within this range.
        Customers who are opting additional features such as streaming movies streaming TV tech support device protection 
        seem to have more density and contribute to not being churned.

        For females most of the customers got churned there monthly charges range between 60 to 100 while for male most 
        of the customers got churned when there monthly charges raised from 60 to 120.

        For monthly charges at 20 units we can see the peak of the graph that indicates customers whether male or female 
        were not getting charned when they had monthly charges near or equal to 20.

USING REGRESSION PLOT CHECKING LINEAR RELATIONSHIP BETWEEN TOTAL CHARGES, MONTHLY CHARGES AND OTHER VARIABLES

        Tenure and monthly charges SIM to have positive relationship with total charges.

        Similarly tenure and total charges have positive relationship with monthly charges.

        Customer being senior citizen or not has more or less neutral relationship with monthly and total charges.

ANALYSING HISTOGRAM FOR FREQUENCIES
        Customer being senior citizen has very less frequency as compared to customer not being a senior citizen.

        For tenure maximum frequency peaks can be seen at range 0 to 10 and 60 to 70.

        For total charges maximum frequency lies in range 0 to 2000.

        For monthly charges maximum frequency peak lies at 20 to 30 and 70 to 90.

ANALYSING DISTRIBUTION PLOT

        By distribution plot for monthly charges total charges tenure is not following normal distribution.

ANALYSING OUTLIERS USING BOX PLOT 
    One outlier can be seen in senior citizen accept no other columns contain outliers.

SKEWNESS VALUES
        Skewness values for all numerical columns are within range and we can conclude that data is not skewed.
        Senior citizen column has highest square value as we saw that it contained outlier.

CORRELATION ANALYSIS AND CHECKING MULTI COLLINEARITY PROBLEM
        Using data.corr function and heatmap we cannot that correlation coefficient values for total charges and 
            tenure column are the same does we need to proceed with checking variance inflation factor.
        
        Values of total charges and tenure are more than 5 does we can control that multi collinearity problem exist.

FURTHER STEPS PERFORMED
        Selection of features and target column
        Encoding categorical columns
        Normalising data using power transform
        Feature selection using principle component analysis
        Splitting training and testing data
        Model instantiation
        Model validation
        Result: 
                        Training accuracy:  82.3253517649963
                Testing accuracy:  78.11375089992801
                Confusion matrix: 
                 [[819 207]
                 [ 97 266]]
                classification report:                precision    recall  f1-score   support

                           0       0.89      0.80      0.84      1026
                           1       0.56      0.73      0.64       363

                    accuracy                           0.78      1389
                   macro avg       0.73      0.77      0.74      1389
                weighted avg       0.81      0.78      0.79      1389
        Selecting best model and hyperparameter boosting
        Saving the model.
        
"""

In [ ]:
# Univariate analysis:

for i in cat_columns:
    f= plt.figure(figsize=(12,5))
    ax = sns.countplot(x=df[i],data=df)
    plt.xticks(rotation=90)

In [ ]:
# PieCharts

In [ ]:
for i in cat_columns:
    df.groupby(i).size().plot(kind='pie',autopct='%.2f')
    plt.xlabel(i)
    plt.show()

In [ ]:
cont_columns

In [ ]:
df['SeniorCitizen'].unique()

In [ ]:
cat_columns

In [ ]:
imp_cols = ['gender', 'Partner', 'Dependents', 'PhoneService',
            'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
            'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
            'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']

In [ ]:
# Checking counts btw rel and avg price by region

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="TotalCharges", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="PaymentMethod", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="PaperlessBilling", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="Contract", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="StreamingMovies", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="StreamingTV", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="TechSupport", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="DeviceProtection", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="OnlineBackup", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="OnlineSecurity", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="InternetService", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="MultipleLines", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="PhoneService", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="Dependents", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="Partner", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="gender", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
AvgPr = []
for i in df.TotalCharges:
    if i < 2000:
        AvgPr.append('Less')
    elif i < 6000 and i > 2000:
        AvgPr.append('Avg')
    else:
        AvgPr.append('High')
        
df['Avg Charges Range'] = AvgPr

In [ ]:
df['Avg Charges Range'].unique()

In [ ]:
df['Avg Charges Range'].nunique()

In [ ]:
df['Avg Charges Range'].value_counts()

In [ ]:
f= plt.figure(figsize=(12,50))
ax = sns.catplot(x="Avg Charges Range", kind="count",hue = 'Churn', data=df)
plt.xticks(rotation=90)
plt.show()

In [ ]:
cat_columns

In [ ]:
Analysis using box plots:

In [ ]:
sns.boxplot(x ='Avg Charges Range', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='PaymentMethod', y ='TotalCharges', data = df, hue ='Churn')
plt.xticks(rotation=90)

In [ ]:
sns.boxplot(x ='PaperlessBilling', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='Contract', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='StreamingMovies', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='StreamingTV', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='TechSupport', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='DeviceProtection', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='OnlineBackup', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='OnlineSecurity', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='InternetService', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='MultipleLines', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='PhoneService', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='Dependents', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='Partner', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
sns.boxplot(x ='gender', y ='TotalCharges', data = df, hue ='Churn')

In [ ]:
# Analysing outliers using violin plot

cols=  ['gender', 'Partner', 'Dependents', 'PhoneService',
       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaperlessBilling', 'PaymentMethod']

In [ ]:
sns.violinplot(x = 'PaymentMethod' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Contract' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'StreamingMovies' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'StreamingTV' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'TechSupport' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'DeviceProtection' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'OnlineBackup' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'OnlineSecurity' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'InternetService' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'MultipleLines' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'PhoneService' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Dependents' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Partner' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'gender' , y ='TotalCharges', data = df, hue ='Churn', split = True)

In [ ]:
# Monthly charges:

In [ ]:
cont_columns

In [ ]:
cols

In [ ]:
sns.violinplot(x = 'gender' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Partner' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Dependents' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'PhoneService' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'MultipleLines' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'InternetService' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'OnlineSecurity' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'DeviceProtection' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'TechSupport' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'StreamingTV' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'StreamingMovies' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'Contract' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'PaperlessBilling' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
sns.violinplot(x = 'PaymentMethod' , y ='MonthlyCharges', data = df, hue ='Churn', split = True)

In [ ]:
cont_columns

In [ ]:
cat_columns

In [ ]:
cols

In [ ]:
# Analysing through strip plots:

In [ ]:
for i in cols:
    sns.stripplot(x = i, y ='MonthlyCharges', data = df,jitter = True, hue ='Churn', dodge = True)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cols:
    sns.stripplot(x = i, y ='TotalCharges', data = df,jitter = True, hue ='Churn', dodge = True)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Now using swarm plot:

for i in cols:
    sns.swarmplot(x = i, y ='MonthlyCharges', data = df, hue ='Churn', dodge = True)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cols:
    sns.swarmplot(x = i, y ='TotalCharges', data = df,hue ='Churn', dodge = True)
    plt.xticks(rotation=90)
    plt.show()

# Analysing numerical columns

In [ ]:
for i in ['SeniorCitizen', 'tenure', 'MonthlyCharges','TotalCharges']:
    sns.scatterplot(x = df.index, y = i, data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in ['SeniorCitizen', 'tenure', 'MonthlyCharges','TotalCharges']:
    sns.lineplot(x = df.index, y = i, data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

# Analysing relationships

In [ ]:
for i in cols:
    sns.relplot(x = i, y = 'MonthlyCharges', data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cols:
    sns.relplot(x = i, y = 'TotalCharges', data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in ['SeniorCitizen', 'tenure', 'MonthlyCharges']:
    sns.relplot(x = i, y = 'TotalCharges', data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in ['SeniorCitizen', 'tenure', 'TotalCharges']:
    sns.relplot(x = i, y = 'MonthlyCharges', data = df,hue='Churn')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
sns.relplot(
    data=df, kind="line",
    x="TotalCharges", y="MonthlyCharges", hue=df['gender']
)

In [ ]:
for i in cont_columns:
    sns.regplot(x = i, y = 'TotalCharges', data = df,scatter_kws={"color": "black"}, line_kws={"color": "red"})
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cont_columns:
    sns.regplot(x = i, y = 'MonthlyCharges', data = df,scatter_kws={"color": "black"}, line_kws={"color": "red"})
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cont_columns:
    sns.lmplot(x = i, y = 'MonthlyCharges',hue='Churn', data=df, palette='Set2', height=8, aspect=1.2)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
for i in cont_columns:
    sns.lmplot(x = i, y = 'TotalCharges',hue='Churn', data=df, palette='Set2', height=8, aspect=1.2)
    plt.xticks(rotation=90)
    plt.show()

# Analysing data frequencies , Outliers and data distribution

In [ ]:
# Histogram plot:

for i in ['SeniorCitizen', 'tenure', 'TotalCharges','MonthlyCharges']:
    plt.figure(figsize=(10,6),facecolor='orange')
    plt.hist(df[i],bins=10)
    plt.xlabel(i)
    plt.show()

In [ ]:
# Checking the distribution of continuous variables:

for i in ['SeniorCitizen', 'tenure', 'TotalCharges','MonthlyCharges']:
    plt.figure(figsize=(12,7),facecolor='orange')
    sns.distplot(df[i])
    plt.show()

In [ ]:
# Checking the outliers in continuous variables:

for i in ['SeniorCitizen', 'tenure', 'TotalCharges','MonthlyCharges']:
    plt.figure(figsize=(12,7),facecolor='orange')
    sns.boxplot(df[i])
    plt.show()

In [ ]:
df.skew().sort_values()

In [ ]:
df.shape

In [ ]:
# Using Z Statistics to check and remove any more outliers:

from scipy.stats import zscore

z_score = zscore(df[cont_columns])

abs_z_score = np.abs(z_score)

filtering_entry = (abs_z_score < 2.5).all(axis=1) # values lying in 3 times std will be removed

df = df[filtering_entry]

df.describe()

In [ ]:
sns.pairplot(df,hue='Churn')

In [ ]:
# Using label encoder to encode dependent variable column:

# Import label encoder
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['Churn']= label_encoder.fit_transform(df['Churn'])
  
df['Churn'].unique()

In [ ]:
# Correlation between features and label:

# Replacing attrition column values:

df.drop(columns = 'Churn',axis = 1).corrwith(df.Churn).plot(kind='bar',grid=True,figsize=(10,7),title='corelation between features and labels')
plt.show()

In [ ]:
# Checking relationship between independent and dependent variable:

df_corr = df.corr().abs()
plt.figure(figsize=(18,14))
sns.heatmap(df_corr,annot=True,annot_kws={'size':10})
plt.show()

In [ ]:
cont_columns = ['SeniorCitizen', 'tenure', 'MonthlyCharges','TotalCharges']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


vif = pd.DataFrame()
vif['vif'] = [variance_inflation_factor(df[cont_columns],i) for i in range(df[cont_columns].shape[1])]
vif['features'] = df[cont_columns].columns
vif

In [ ]:
cont_columns1 = ['SeniorCitizen', 'MonthlyCharges','tenure']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


vif = pd.DataFrame()
vif['vif'] = [variance_inflation_factor(df[cont_columns1],i) for i in range(df[cont_columns1].shape[1])]
vif['features'] = df[cont_columns1].columns
vif

In [ ]:
# Splitting data into features and label:

y = df['Churn']
X_old = df.drop(['Churn','TotalCharges'],axis=1)

In [ ]:
X_old.columns

In [ ]:
X = pd.get_dummies(X_old)

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt = PowerTransformer()

X_trans = pt.fit_transform(X)

In [ ]:
X_trans

In [ ]:
from sklearn.decomposition import PCA
# Using PCA i.e. Principal Component Analysis that is a diamensionallity reduction technique:

pca = PCA()
pca.fit_transform(X_trans)

In [ ]:
# Using Scree Plot to identify best components:

plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Principal Components')
plt.ylabel('Variance Covered')
plt.title('PCA')
plt.show()


In [ ]:
# Splitting our data to training data and testing data
# x_train,x_test,y_train,y_test

X_train,X_test,y_train,y_test = train_test_split(X_trans,y,test_size=0.20,random_state=1)

# Here we are keeping training data as our scalled data and testing data as our label or target.

In [ ]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

In [ ]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

In [ ]:
arr = np.array(y_train_res)

un, co = np.unique(arr,return_counts=True)

dict(zip(un,co))

In [ ]:
# Logistic Regression:

log_reg = LogisticRegression(random_state=1)

log_reg.fit(X_train_res,y_train_res) 

pred_train = log_reg.predict(X_train_res)

y_pred = log_reg.predict(X_test)

acc = accuracy_score(y_test,y_pred)

confusion_mat = confusion_matrix(y_test,y_pred)

print('Confusion matrix: \n',confusion_mat)

clr = classification_report(y_test,y_pred)

print('classification report: ' ,clr)

In [ ]:
# Best result for Decision Tree Classifier:

dtc = DecisionTreeClassifier(random_state=1)
dtc.fit(X_train_res,y_train_res) 
pred_train_dtc = dtc.predict(X_train_res)
y_pred = dtc.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_dtc)*100)
print('Testing accuracy: ', acc*100)
confusion_mat = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',confusion_mat)
clr = classification_report(y_test,y_pred)
print('classification report: ' ,clr)

In [ ]:
# Random forrest classifier model:

rfc_f = RandomForestClassifier()
rfc_f.fit(X_train_res,y_train_res) 
pred_train_rfc_f = rfc_f.predict(X_train_res)
y_pred = rfc_f.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_rfc_f)*100)
print('Testing accuracy: ', acc*100)
confusion_mat = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',confusion_mat)
clr = classification_report(y_test,y_pred)
print('classification report: ' ,clr)

In [ ]:
# Using best parameters for improved score:

svc = SVC()

svc.fit(X_train_res,y_train_res)

pred_train_svc = svc.predict(X_train_res)
y_pred = svc.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_svc)*100)
print('Testing accuracy: ', acc*100)
confusion_mat = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',confusion_mat)
clr = classification_report(y_test,y_pred)
print('classification report: ' ,clr)

In [ ]:
# KNN classifier:

knn = KNeighborsClassifier()
knn.fit(X_train_res,y_train_res) 
pred_train_knn = knn.predict(X_train_res)
y_pred = knn.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_knn)*100)
print('Testing accuracy: ', acc*100)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# GBC

gbdt_clf = GradientBoostingClassifier(random_state=3)
gbdt_clf.fit(X_train_res,y_train_res) 
pred_train_gbdt_clf = gbdt_clf.predict(X_train_res)
y_pred = gbdt_clf.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_gbdt_clf)*100)
print('Testing accuracy: ', acc*100)
confusion_mat = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',confusion_mat)
clr = classification_report(y_test,y_pred)
print('classification report: ' ,clr)

In [ ]:
from sklearn.model_selection import KFold , cross_val_score
k_f = KFold(n_splits =3)
k_f

In [ ]:
for train,test in k_f.split([1,2,3,4,5,6,7,8,9,10]):
    print('train:',train,'test:',test)

In [ ]:
# Cross Validation score to check if model is overfitting:

cross_val_score(gbdt_clf,X_train_res,y_train_res,cv=5) # 

In [ ]:
cross_val_score(gbdt_clf,X_train_res,y_train_res,cv=5).mean()

In [ ]:
# ADA model:
from sklearn.ensemble import AdaBoostClassifier

ada_boosted = AdaBoostClassifier()
ada_boosted.fit(X_train_res,y_train_res)
yb_pred = ada_boosted.predict(X_test)
pred_train_ada = ada_boosted.predict(X_train_res)
y_pred = ada_boosted.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print('Training accuracy: ', accuracy_score(y_train_res,pred_train_ada)*100)
print('Testing accuracy: ', acc*100)
confusion_mat = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',confusion_mat)
clr = classification_report(y_test,y_pred)
print('classification report: ' ,clr)

In [ ]:
# HyperParameter tuning using Randomised Search CV :

from sklearn.model_selection import RandomizedSearchCV

params = {'n_estimators':[47,50,60,70],'learning_rate':[0.25,0.3,0.4]}

rnd = RandomizedSearchCV(ada_boosted,param_distributions=params)

rnd.fit(X_train_res,y_train_res)

In [ ]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_train_res, y_train_res)
gnb_predictions = gnb.predict(X_test)
  
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)
print(accuracy)

In [ ]:
# Tuning best model

In [ ]:
# HYPER PARAMETER TUNING:

# Tuning parameters using GridSearchCV:

params = {'max_depth':range(4,8),
          'min_samples_split':range(2,8,2),
          'learning_rate':np.arange(0.1,0.3)} # at which rate our model should learn

grd = GridSearchCV(gbdt_clf,param_grid=params)

grd.fit(X_train_res,y_train_res)

In [ ]:
grd.best_params_

In [ ]:
# Instantiating gradient boosting classifier:

gbdt_clf_f = GradientBoostingClassifier(learning_rate=0.1, max_depth=7, min_samples_split=2)

# Training the model
gbdt_clf_f.fit(x_train,y_train)

In [ ]:
# Creating pipeline:

from sklearn.pipeline import Pipeline

pipe1 = Pipeline([('pt',PowerTransformer()),
                  ('pca',PCA(n_components=17000)),('base_model1',RandomForestRegressor(n_estimators = 10, random_state = 42))])

pipe1.fit(X_train_res,y_train_res)

y_pred = pipe.predict(X_test)

accuracy_score(y_test,y_pred)

# Saving regression model to pickle string

import pickle 
saved_model1 = pickle.dumps(pipe1) 
pipe_pickle1 = pickle.loads(saved_model1)
pipe_pickle1.predict(X_test) # predicting testing data

